In [3]:
from fastapi import FastAPI
from gdeltdoc import GdeltDoc, Filters
import pandas as pd
import os
import datetime

In [2]:
def descargar_datos_diarios(funcion_obtener_datos: str, keyword: str, country: str,descarga: str):   
    f = Filters(keyword=keyword,
                    start_date="2017-01-01",
                    end_date="2023-12-31",
                    country=country)
    gd = GdeltDoc() 
    if funcion_obtener_datos.lower() == "tone":
        results = gd.timeline_search("timelinetone", f)
        df = pd.DataFrame(results)
    elif funcion_obtener_datos.lower() == "popularity":
        results = gd.timeline_search("timelinevol", f)
        df = pd.DataFrame(results)

    else:
        return "Error: La función de extracción de datos especificada no es válida."

    if descarga.lower() == "yes": # para guardar el archivo si el parametro descarga es YES
        carpeta_padre = "data_diario" # Creamos la carpeta padre "data_diario" si no existe
        if not os.path.exists(carpeta_padre):
            os.makedirs(carpeta_padre)
    
        carpeta_tipo_datos = f"{carpeta_padre}/{funcion_obtener_datos}" # luego la ruta de la segunda carpeta
        if not os.path.exists(carpeta_tipo_datos):
            os.makedirs(carpeta_tipo_datos)
    
        # y despues la del archivo csv
        file_number = 1
        while os.path.exists(f"{carpeta_tipo_datos}/csv_{funcion_obtener_datos}_{country}_{file_number}.csv"):
            file_number += 1
        filename = f"{carpeta_tipo_datos}/csv_{funcion_obtener_datos}_{country}_diarios_{file_number}.csv"
        df.to_csv(filename, index=False)
    
    return df

In [25]:
descargar_datos_diarios("popularity","Health service delivery", "BE","yes")

KeyError: 'timeline'

In [7]:
descargar_datos_diarios("tone","Inclusive growth", "Spain","yes")

,datetime,Average Tone
0,2017-01-01 00:00:00+00:00,0.0000
1,2017-01-02 00:00:00+00:00,-4.1282
2,2017-01-03 00:00:00+00:00,0.0000
3,2017-01-04 00:00:00+00:00,0.0000
4,2017-01-05 00:00:00+00:00,0.0000
...,...,...
2547,2023-12-27 00:00:00+00:00,0.0000
2548,2023-12-28 00:00:00+00:00,0.0000
2549,2023-12-29 00:00:00+00:00,0.0000
2550,2023-12-30 00:00:00+00:00,0.0000


In [4]:
def descargar_datos_mensuales(funcion_obtener_datos: str, keyword: str, country: str):
    if funcion_obtener_datos == "tone":
        df = descargar_datos_diarios(funcion_obtener_datos, keyword, country, "no") # para que no se descarguen diarios
        df['datetime'] = pd.to_datetime(df['datetime'])
        # Agrupamos los datos por mes y calculamos la suma de los valores para cada mes
        df_monthly = df.groupby(pd.Grouper(key='datetime', freq='M')).sum().reset_index()
    elif funcion_obtener_datos == "popularity":
        df = descargar_datos_diarios(funcion_obtener_datos, keyword, country,"no")
        df['datetime'] = pd.to_datetime(df['datetime'])
        df_monthly = df.groupby(pd.Grouper(key='datetime', freq='M')).sum().reset_index()

    else:
        return {"mensaje": "Función de obtener datos no válida"}

    carpeta_padre = "data_mensual" # Creamos la carpeta padre "data_limpio" si no existe
    if not os.path.exists(carpeta_padre):
        os.makedirs(carpeta_padre)
    
    carpeta_tipo_datos = f"{carpeta_padre}/{funcion_obtener_datos}" # luego la ruta de la segunda carpeta
    if not os.path.exists(carpeta_tipo_datos):
        os.makedirs(carpeta_tipo_datos)
    
    # y despues la del archivo csv
    file_number = 1
    while os.path.exists(f"{carpeta_tipo_datos}/csv_{funcion_obtener_datos}_{country}_{file_number}.csv"):
        file_number += 1
    filename = f"{carpeta_tipo_datos}/csv_{funcion_obtener_datos}_{country}_mensual_{file_number}.csv"
    df_monthly.to_csv(filename, index=False)
    
    return {"mensaje": f"Datos de {funcion_obtener_datos} guardados exitosamente en '{filename}'"}

In [11]:
descargar_datos_mensuales("tone", "Inclusive Growth", "Spain")

{'mensaje': "Datos de tone guardados exitosamente en 'data_mensual/tone/csv_tone_Spain_mensual_1.csv'"}

In [12]:
descargar_datos_mensuales("popularity", "Inclusive Growth", "Spain")

{'mensaje': "Datos de popularity guardados exitosamente en 'data_mensual/popularity/csv_popularity_Spain_mensual_1.csv'"}

In [5]:
def descargar_datos_trimestrales(funcion_obtener_datos: str, keyword: str, country: str):
    if funcion_obtener_datos == "tone":
        df = descargar_datos_diarios(funcion_obtener_datos, keyword, country, "no")
        df['datetime'] = pd.to_datetime(df['datetime'])
        # Agrupamos los datos por trimestre y calculamos la suma de los valores para cada trimestre
        df_quaterly = df.groupby(pd.Grouper(key='datetime', freq='Q')).sum().reset_index()
    elif funcion_obtener_datos == "popularity":
        df = descargar_datos_diarios(funcion_obtener_datos, keyword, country, "no")
        df['datetime'] = pd.to_datetime(df['datetime'])
        df_quaterly = df.groupby(pd.Grouper(key='datetime', freq='Q')).sum().reset_index()
    else:
        return {"mensaje": "Función de obtener datos no válida"}

    carpeta_padre = "data_trimestral" # Creamos la carpeta padre "data_limpio" si no existe
    if not os.path.exists(carpeta_padre):
        os.makedirs(carpeta_padre)
    
    carpeta_tipo_datos = f"{carpeta_padre}/{funcion_obtener_datos}_limpios" # luego la ruta de la segunda carpeta
    if not os.path.exists(carpeta_tipo_datos):
        os.makedirs(carpeta_tipo_datos)
    
    # y despues la del archivo csv
    file_number = 1
    while os.path.exists(f"{carpeta_tipo_datos}/csv_{funcion_obtener_datos}_{country}_limpios_{file_number}.csv"):
        file_number += 1
    filename = f"{carpeta_tipo_datos}/csv_{funcion_obtener_datos}_{country}_{file_number}.csv"
    df_quaterly.to_csv(filename, index=False)
    
    return f"Mean of {funcion_obtener_datos.capitalize()} data downloaded successfully"


In [15]:
descargar_datos_trimestrales("tone", "Inclusive Growth", "Spain")

'Mean of Tone data downloaded successfully'

In [16]:
descargar_datos_trimestrales("popularity", "Inclusive Growth", "Spain")

'Mean of Popularity data downloaded successfully'

In [6]:
def limpiar_dataframe(df: pd.DataFrame,freq:str): # PARA EL TIPO DE FRECUENCIA QUE QUEREMOS, DIARIO, MENSUAL O TRIMESTRAL
    # Nos aseguramos que el DataFrame tenga todas las fechas. Si alguna no está, rellenamos con 0.
    start_date = df['datetime'].min()
    end_date = df['datetime'].max()
    all_dates = pd.date_range(start=start_date, end=end_date, freq=freq)
    df = df.set_index('datetime').reindex(all_dates).fillna(0).reset_index()
    df = df.rename(columns={'index': 'datetime'}) # Renombramos la columna del índice
    df_limpio= df.dropna().drop_duplicates()
    return df_limpio 

In [7]:
def endpoint_limpiar_dataframe(funcion_obtener_datos: str, keyword: str, country: str,time:str): # LOS PARAMETROS NECESARIOS
    if funcion_obtener_datos.lower() == "tone":
        results = descargar_datos_diarios(funcion_obtener_datos, keyword, country, "no")
        df = pd.DataFrame(results)
        folder_name = "tono"
    elif funcion_obtener_datos.lower() == "popularity":
        results = descargar_datos_diarios(funcion_obtener_datos, keyword, country, "no")
        df = pd.DataFrame(results)
        folder_name = "popularity"
    else:
        return "Error: La función de extracción de datos especificada no es válida."
        
    if time.lower() == "mensual":
        df['datetime'] = pd.to_datetime(df['datetime'])
        df_monthly = df.groupby(pd.Grouper(key='datetime', freq='M')).sum().reset_index()
        save_folder = f"./data_limpio/{folder_name}/mensual"
        df_save = df_monthly
        df_save = limpiar_dataframe(df_save, "M") # LIMPIEZA MENSUAL
    elif time.lower() == "diary":
        save_folder = f"./data_limpio/{folder_name}/diario"
        df_save = limpiar_dataframe(df,"D")
    elif time.lower() == "quaterly":
        df['datetime'] = pd.to_datetime(df['datetime'])
        df_trimestral = df.groupby(pd.Grouper(key='datetime', freq='Q')).sum().reset_index() 
        save_folder = f"./data_limpio/{folder_name}/quaterly"
        df_save = df_trimestral
        df_save = limpiar_dataframe(df_save,"Q") # LIMPIEZA EN TRIMESTRES
            
    else:
        return "Error: El intervalo especificado no es válido."
        
    os.makedirs(save_folder, exist_ok=True) # para asegurarnos de que la carpeta este

    # y guardamos
    file_timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S") # para evitar sobreescribir
    filename = f"{save_folder}/{keyword}_csv_limpio_{folder_name}_{time}_{country}_{file_timestamp}.csv"  # Agregar el nombre del país y el periodo de tiempo al archivo CSV
    df_save.to_csv(filename, index=False)
    
    return f"Mean of {funcion_obtener_datos.capitalize()} data downloaded successfully"

In [20]:
endpoint_limpiar_dataframe("tone","Inclusive Growth", "Spain","mensual")

'Mean of Tone data downloaded successfully'

In [24]:
endpoint_limpiar_dataframe("tone","Inclusive Growth", "Spain","diary")

'Mean of Tone data downloaded successfully'

In [27]:
endpoint_limpiar_dataframe("tone","Inclusive Growth", "Spain","quaterly")

'Mean of Tone data downloaded successfully'

In [28]:
endpoint_limpiar_dataframe("popularity","Inclusive Growth", "Spain","quaterly")

'Mean of Popularity data downloaded successfully'

In [31]:
endpoint_limpiar_dataframe("popularity","Inclusive Growth", "Spain","mensual")

'Mean of Popularity data downloaded successfully'

In [32]:
endpoint_limpiar_dataframe("popularity","Inclusive Growth", "Spain","diary")

'Mean of Popularity data downloaded successfully'

In [8]:
def extraccion_total(funcion_obtener_datos: str, keyword: str, time: str):
    countries = ['Austria', 'Belgium', 'Switzerland', 'Cyprus', 'EZ', 'Germany', 'Denmark', 'Greece', 'Spain', 'Finland', 'France', 'Ireland', 'Italy', 'Luxembourg', 'Netherlands', 'Norway', 'Portugal', 'Sweden', 'Slovenia']
    for country in countries:
        f = Filters(keyword=keyword,
                    start_date="2017-01-01",
                    end_date="2023-12-31",
                    country=country)
        gd = GdeltDoc() 
        
        if funcion_obtener_datos.lower() == "tone":
            results = gd.timeline_search("timelinetone", f)
            df = pd.DataFrame(results)
            folder_name = "tone"
        elif funcion_obtener_datos.lower() == "popularity":
            results = gd.timeline_search("timelinevol", f)
            df = pd.DataFrame(results)
            folder_name = "popularity"
        else:
            return "Error: La función de extracción de datos especificada no es válida."
        
        if time.lower() == "monthly":
            df['datetime'] = pd.to_datetime(df['datetime'])
            df_monthly = df.groupby(pd.Grouper(key='datetime', freq='M')).sum().reset_index()
            save_folder = f"./data_final/{folder_name}/monthly"
            df_save = df_monthly
            df_save = limpiar_dataframe(df_save, "M") # LIMPIEZA MENSUAL
        elif time.lower() == "quaterly":
            df['datetime'] = pd.to_datetime(df['datetime'])
            df_trimestral = df.groupby(pd.Grouper(key='datetime', freq='Q')).sum().reset_index() 
            save_folder = f"./data_final/{folder_name}/quaterly"
            df_save = df_trimestral
            df_save = limpiar_dataframe(df_save,"Q") # LIMPIEZA EN TRIMESTRES
            
        else:
            return "Error: El intervalo especificado no es válido."
        
        os.makedirs(save_folder, exist_ok=True) # para asegurarnos de que la carpeta este

        # y guardamos
        file_timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S") # para evitar sobreescribir
        filename = f"{save_folder}/{keyword}_{folder_name}_{time}_{country}_{file_timestamp}.csv"  # Agregar el nombre del país y el periodo de tiempo al archivo CSV
        df_save.to_csv(filename, index=False)
    
    return f"Mean of {funcion_obtener_datos.capitalize()} data downloaded successfully for all countries"


In [20]:
extraccion_total("tone", "Inclusive Growth", "quaterly")

'Mean of Tone data downloaded successfully for all countries'

In [21]:
extraccion_total("tone", "Inclusive Growth", "monthly")

'Mean of Tone data downloaded successfully for all countries'

In [22]:
extraccion_total("popularity", "Inclusive Growth", "quaterly")

'Mean of Popularity data downloaded successfully for all countries'

In [23]:
extraccion_total("popularity", "Inclusive Growth", "monthly")

'Mean of Popularity data downloaded successfully for all countries'

In [11]:
def calcular_media_columna_total(csv_folder, column_name):
    """
    Calcula la media de una columna específica en todos los archivos CSV en una carpeta y guarda los resultados en un archivo CSV.

    Parámetros:
        csv_folder (str): La ruta al directorio que contiene los archivos CSV.
        column_name (str): El nombre de la columna para calcular la media.

    Retorna:
        pd.DataFrame: DataFrame con las medias de la columna específica.
    """
    dfs = []
    for filename in os.listdir(csv_folder):
        if filename.endswith(".csv"):
            file_path = os.path.join(csv_folder, filename)
            df = pd.read_csv(file_path)
            print(f"Leyendo archivo: {file_path}")
            if column_name in df.columns:
                media_columna = df[column_name].mean()
                dfs.append(pd.DataFrame({column_name: [media_columna]}))
            else:
                print(f"La columna '{column_name}' no se encontró en el archivo: {file_path}")

    if not dfs:
        print(f"No se encontraron archivos CSV en el directorio: {csv_folder}")
        return pd.DataFrame()

    combined_df = pd.concat(dfs, ignore_index=True)
    return combined_df

# Rutas de las carpetas donde se guardan los archivos CSV de tono y popularidad (mensual y trimestral)
tono_mensual_csv_folder = "./data_final/tone/monthly"
popularity_mensual_csv_folder = "./data_final/popularity/monthly"
tono_trimestral_csv_folder = "./data_final/tone/quaterly"
popularity_trimestral_csv_folder = "./data_final/popularity/quaterly"

# Calcular la media de las columnas específicas para tono y popularidad (mensual y trimestral)
media_tono_mensual = calcular_media_columna_total(tono_mensual_csv_folder, 'Average Tone')
media_popularity_mensual = calcular_media_columna_total(popularity_mensual_csv_folder, 'Volume Intensity')
media_tono_trimestral = calcular_media_columna_total(tono_trimestral_csv_folder, 'Average Tone')
media_popularity_trimestral = calcular_media_columna_total(popularity_trimestral_csv_folder, 'Volume Intensity')

# Guardar los DataFrames como archivos CSV
media_tono_mensual.to_csv("./data_final/media_tono_mensual.csv", index=False)
media_popularity_mensual.to_csv("./data_final/media_popularity_mensual.csv", index=False)
media_tono_trimestral.to_csv("./data_final/media_tono_trimestral.csv", index=False)
media_popularity_trimestral.to_csv("./data_final/media_popularity_trimestral.csv", index=False)

print("Archivos de medias guardados exitosamente.")
# error, ME ELIMINABA LA COLUMNA DE DATETIME, Y TANTO MENSUAL COMO TRIMESTRAL TENIA 2O REGISTROS

Leyendo archivo: ./data_final/tone/monthly\Inclusive Growth_tone_monthly_Austria_20240407161138.csv
Leyendo archivo: ./data_final/tone/monthly\Inclusive Growth_tone_monthly_Belgium_20240407161139.csv
Leyendo archivo: ./data_final/tone/monthly\Inclusive Growth_tone_monthly_Cyprus_20240407161142.csv
Leyendo archivo: ./data_final/tone/monthly\Inclusive Growth_tone_monthly_Denmark_20240407161146.csv
Leyendo archivo: ./data_final/tone/monthly\Inclusive Growth_tone_monthly_EZ_20240407161143.csv
Leyendo archivo: ./data_final/tone/monthly\Inclusive Growth_tone_monthly_Finland_20240407161151.csv
Leyendo archivo: ./data_final/tone/monthly\Inclusive Growth_tone_monthly_France_20240407161152.csv
Leyendo archivo: ./data_final/tone/monthly\Inclusive Growth_tone_monthly_Germany_20240407161145.csv
Leyendo archivo: ./data_final/tone/monthly\Inclusive Growth_tone_monthly_Greece_20240407161148.csv
Leyendo archivo: ./data_final/tone/monthly\Inclusive Growth_tone_monthly_Ireland_20240407161154.csv
Leyendo 

In [13]:
# otra prueba mas
def calcular_media_columna_total(csv_folder, column_name):
    """
    Calcula la media de una columna específica en todos los archivos CSV en una carpeta y guarda los resultados en un archivo CSV.

    Parámetros:
        csv_folder (str): La ruta al directorio que contiene los archivos CSV.
        column_name (str): El nombre de la columna para calcular la media.

    Retorna:
        pd.DataFrame: DataFrame con las medias de la columna específica.
    """
    dfs = []
    for filename in os.listdir(csv_folder):
        if filename.endswith(".csv"):
            file_path = os.path.join(csv_folder, filename)
            df = pd.read_csv(file_path, parse_dates=['datetime'])
            print(f"Leyendo archivo: {file_path}")
            if column_name in df.columns:
                df.set_index('datetime', inplace=True)
                dfs.append(df[column_name].rename(filename.split('.')[0]))  # Renombrar la serie con el nombre del archivo
            else:
                print(f"La columna '{column_name}' no se encontró en el archivo: {file_path}")

    if not dfs:
        print(f"No se encontraron archivos CSV en el directorio: {csv_folder}")
        return pd.DataFrame()

    combined_df = pd.concat(dfs, axis=1)
    media_columna = combined_df.mean(axis=1)  # Calcular la media a lo largo de las columnas
    media_df = pd.DataFrame({column_name: media_columna})
    return media_df

# Rutas de las carpetas donde se guardan los archivos CSV de tono y popularidad (mensual y trimestral)
tono_mensual_csv_folder = "./data_final/tone/monthly"
popularity_mensual_csv_folder = "./data_final/popularity/monthly"
tono_trimestral_csv_folder = "./data_final/tone/quaterly"
popularity_trimestral_csv_folder = "./data_final/popularity/quaterly"

# Calcular las medias de las columnas específicas para tono y popularidad (mensual y trimestral)
media_tono_mensual = calcular_media_columna_total(tono_mensual_csv_folder, 'Average Tone')
media_popularity_mensual = calcular_media_columna_total(popularity_mensual_csv_folder, 'Volume Intensity')
media_tono_trimestral = calcular_media_columna_total(tono_trimestral_csv_folder, 'Average Tone')
media_popularity_trimestral = calcular_media_columna_total(popularity_trimestral_csv_folder, 'Volume Intensity')

# Guardar los DataFrames como archivos CSV
media_tono_mensual.to_csv("./data_final/media_tono_mensual.csv", index=True)
media_popularity_mensual.to_csv("./data_final/media_popularity_mensual.csv", index=True)
media_tono_trimestral.to_csv("./data_final/media_tono_trimestral.csv", index=True)
media_popularity_trimestral.to_csv("./data_final/media_popularity_trimestral.csv", index=True)

print("Archivos de medias guardados exitosamente.")




Leyendo archivo: ./data_final/tone/monthly\Inclusive Growth_tone_monthly_Austria_20240407161138.csv
Leyendo archivo: ./data_final/tone/monthly\Inclusive Growth_tone_monthly_Belgium_20240407161139.csv
Leyendo archivo: ./data_final/tone/monthly\Inclusive Growth_tone_monthly_Cyprus_20240407161142.csv
Leyendo archivo: ./data_final/tone/monthly\Inclusive Growth_tone_monthly_Denmark_20240407161146.csv
Leyendo archivo: ./data_final/tone/monthly\Inclusive Growth_tone_monthly_EZ_20240407161143.csv
Leyendo archivo: ./data_final/tone/monthly\Inclusive Growth_tone_monthly_Finland_20240407161151.csv
Leyendo archivo: ./data_final/tone/monthly\Inclusive Growth_tone_monthly_France_20240407161152.csv
Leyendo archivo: ./data_final/tone/monthly\Inclusive Growth_tone_monthly_Germany_20240407161145.csv
Leyendo archivo: ./data_final/tone/monthly\Inclusive Growth_tone_monthly_Greece_20240407161148.csv
Leyendo archivo: ./data_final/tone/monthly\Inclusive Growth_tone_monthly_Ireland_20240407161154.csv
Leyendo 

In [29]:
def extraccion_total(funcion_obtener_datos: str, time: str):
    countries = ['Austria', 'Belgium', 'Switzerland', 'Cyprus', 'EZ', 'Germany', 'Denmark', 'Greece', 'Spain', 'Finland', 'France', 'Ireland', 'Italy', 'Luxembourg', 'Netherlands', 'Norway', 'Portugal', 'Sweden', 'Slovenia']
    keywords = ["Inclusive growth", "Enterprise Development", "Financial Inclusion", "Social inclusion", "Access to Education", "Health service delivery"]
    for country in countries:
        for keyword in keywords:  # PARA TENER UNA CARPETA POR TOPIC
            print(f"Descargando datos para el país {country} y el tema {keyword}...")
            f = Filters(keyword=keyword,
                        start_date="2017-01-01",
                        end_date="2023-12-31",
                        country=country)
            gd = GdeltDoc()

            try:
                if funcion_obtener_datos.lower() == "tone":
                    results = gd.timeline_search("timelinetone", f)
                    df = pd.DataFrame(results)
                    folder_name = "tone"
                elif funcion_obtener_datos.lower() == "popularity":
                    results = gd.timeline_search("timelinevol", f)
                    df = pd.DataFrame(results)
                    folder_name = "popularity"
                else:
                    return "Error: La función de extracción de datos especificada no es válida."
                
                if df.empty:
                    print("El DataFrame está vacío, pasando al siguiente país y tema...")
                    continue  # Salta a la siguiente iteración si el DataFrame está vacío

                df['datetime'] = pd.to_datetime(df['datetime'])
                if time.lower() == "monthly":
                    df_monthly = df.groupby(pd.Grouper(key='datetime', freq='M')).sum().reset_index()
                    save_folder = f"./data_final/{folder_name}/monthly/{keyword}"  # Carpeta mensual específica para cada tema
                    df_save = df_monthly
                    df_save = limpiar_dataframe(df_save, "M")  # LIMPIEZA MENSUAL
                elif time.lower() == "quaterly":
                    df_trimestral = df.groupby(pd.Grouper(key='datetime', freq='Q')).sum().reset_index()
                    save_folder = f"./data_final/{folder_name}/quaterly/{keyword}"  # Carpeta trimestral específica para cada tema
                    df_save = df_trimestral
                    df_save = limpiar_dataframe(df_save, "Q")  # LIMPIEZA EN TRIMESTRES
                else:
                    return "Error: El intervalo especificado no es válido."

                os.makedirs(save_folder, exist_ok=True)  # para asegurarnos de que la carpeta esté creada
                print(f"Guardando el archivo CSV en {save_folder}...")
                # y guardamos
                file_timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")  # para evitar sobreescribir
                filename = f"{save_folder}/{keyword}_{folder_name}_{time}_{country}_{file_timestamp}.csv"  # Agregar el nombre del país y el periodo de tiempo al archivo CSV
                df_save.to_csv(filename, index=False)
                time.sleep(15)  # Agregar un retraso de 15 segundos entre cada consulta
            except Exception as e:
                print(f"Se produjo un error: {e}")
                continue  # Continúa con la siguiente iteración si se produce una excepción

    return f"Mean of {funcion_obtener_datos.capitalize()} data downloaded successfully for all countries"

In [30]:
extraccion_total("tone", "monthly")
# error DEMASIADAS LLAMADAS, HAY QUE PONER TIEMPO DE ESPERA

Descargando datos para el país Austria y el tema Inclusive growth...
Se produjo un error: The gdelt api returned a non-successful statuscode. This is the response message: Please limit requests to one every 5 seconds or contact kalev.leetaru5@gmail.com for larger queries.


Descargando datos para el país Austria y el tema Enterprise Development...
Se produjo un error: The gdelt api returned a non-successful statuscode. This is the response message: Please limit requests to one every 5 seconds or contact kalev.leetaru5@gmail.com for larger queries.


Descargando datos para el país Austria y el tema Financial Inclusion...
Se produjo un error: The gdelt api returned a non-successful statuscode. This is the response message: Please limit requests to one every 5 seconds or contact kalev.leetaru5@gmail.com for larger queries.


Descargando datos para el país Austria y el tema Social inclusion...
Se produjo un error: The gdelt api returned a non-successful statuscode. This is the response mes

'Mean of Tone data downloaded successfully for all countries'